In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "NN_keras"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [3]:
allPlayers = pd.read_csv('./processeddata/season_stats.csv', skipinitialspace=True)
allPlayers

,G,Pos,PER,TS%,FTr,TRB%,AST%,STL%,BLK%,TOV%,...,SPG,BPG,name,MVP Awards,1st Team,2nd Team,3rd Team,All-NBA Teams,All Star Selections,isHOF
0,1361,2.888889,13.872222,0.543556,0.437111,14.655556,5.216667,1.438889,0.850000,11.488889,...,0.810434,0.412932,A.C. Green,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,8,4.000000,-0.400000,0.190000,0.000000,20.100000,0.000000,0.800000,0.000000,12.500000,...,0.125000,0.000000,A.J. Bramlett,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,151,1.000000,11.550000,0.480000,0.241000,5.550000,15.850000,0.900000,0.450000,11.950000,...,0.377483,0.158940,A.J. English,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80,0.000000,4.366667,0.324000,0.068333,2.533333,23.466667,2.466667,0.500000,14.433333,...,0.250000,0.150000,A.J. Guyton,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22,4.000000,8.400000,0.472000,0.476000,12.800000,3.800000,0.300000,7.200000,16.400000,...,0.045455,0.590909,A.J. Hammons,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3915,570,4.000000,16.462500,0.525000,0.425875,10.318058,8.339991,1.654735,1.384296,15.085099,...,0.568911,0.351389,Zelmo Beaty,0.0,0.0,0.0,0.0,0.0,5.0,1.0
3916,146,3.875000,13.400000,0.493375,1.132000,15.162500,1.625000,4.187500,0.475000,15.400000,...,0.294521,0.191781,Zendon Hamilton,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3917,32,1.000000,8.433333,0.412667,0.211000,7.866667,11.366667,1.033333,0.000000,10.433333,...,0.125000,0.000000,Zoran Dragic,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3918,148,0.333333,9.800000,0.492000,0.401667,7.433333,18.233333,1.966667,0.333333,17.500000,...,0.391892,0.047297,Zoran Planinic,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
allPlayers = allPlayers.set_index('name')

In [5]:
futureHOF = ["LeBron James", "Chris Paul", "Kevin Durant", "Russell Westbrook",
             "James Harden", "Stephen Curry", "Dwight Howard", "Carmelo Anthony",
             "Vince Carter", "Pau Gasol", "Chris Bosh", "Anthony Davis", "Kyle Lowry", "Dwyane Wade",
             "Dirk Nowitzki", "Kevin Love","Paul Pierce", "Damian Lillard", "Paul George", "Kyrie Irving",
             "Blake Griffin", "Draymond Green","Kawhi Leonard", "Klay Thompson", "LaMarcus Aldridge", "Jimmy Butler",
             "Rajon Rondo","John Wall", "Giannis Antetokounmpo", "Kemba Walker", "Al Horford", "Derrick Rose",
             "Tony Parker", "Chauncey Billups", "Shawn Marion", "Joe Johnson", "Shawn Kemp", "Bill Laimbeer", "Gilbert Arenas"]
allPlayers = allPlayers.drop(futureHOF)

In [6]:
nonHOF_indices = allPlayers[allPlayers.isHOF == 0].index
random_indices = np.random.choice(nonHOF_indices, 3261, replace=False)
allPlayers = allPlayers.drop(random_indices)
allPlayers

,G,Pos,PER,TS%,FTr,TRB%,AST%,STL%,BLK%,TOV%,...,APG,SPG,BPG,MVP Awards,1st Team,2nd Team,3rd Team,All-NBA Teams,All Star Selections,isHOF
name,,,,,,,,,,,,,,,,,,,,,
A.J. Bramlett,8,4.000000,-0.400000,0.190000,0.000000,20.100000,0.000000,0.800000,0.000000,12.500000,...,0.000000,0.125000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aaron Williams,838,3.500000,11.120000,0.532250,0.346850,13.930000,5.155000,1.165000,3.905000,17.605000,...,0.626492,0.381862,0.712411,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acie Law,265,0.000000,11.122222,0.507889,0.510000,4.444444,18.111111,2.133333,0.366667,16.533333,...,1.581132,0.475472,0.018868,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adam Keefe,617,3.222222,12.244444,0.544222,0.535556,14.544444,6.255556,1.544444,1.333333,17.266667,...,0.692058,0.497569,0.280389,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adrian Dantley,1107,2.000000,20.373684,0.600053,0.565105,8.510526,12.489474,1.326316,0.263158,12.499216,...,2.939476,0.998193,0.168925,0.0,0.0,2.0,0.0,2.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Worthy Patterson,4,1.000000,11.000000,0.394000,0.250000,9.949210,13.009962,1.648269,1.410624,15.085099,...,0.500000,9.974263,6.117565,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Xavier McDaniel,951,2.142857,14.964286,0.505071,0.253643,11.721429,10.642857,1.514286,0.985714,13.157143,...,2.063091,0.911672,0.485804,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Yao Ming,486,4.000000,22.850000,0.593375,0.460500,16.512500,9.475000,0.562500,4.587500,14.337500,...,1.582305,0.388889,1.893004,0.0,0.0,2.0,3.0,5.0,8.0,1.0


In [7]:
feature_columns = ['G', 'PER', 'TS%', 'FTr', 'TRB%', 'AST%', 
                   'STL%', 'BLK%', 'TOV%', 'USG%', 'WS', 'BPM',
                   'VORP', 'FG', 'FGA', 'FG%','3P', '3PA', '3P%',
                   'FT', 'FTA','RPG', 'APG', 'SPG', 'BPG', 
                   'PPG', 'MVP Awards', '1st Team', '2nd Team', '3rd Team', 
                   'All-NBA Teams', 'All Star Selections']
X = allPlayers[feature_columns]
y = allPlayers['isHOF']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
for node in [1, 5, 10, 20, 25]:
    # training the data
    clf = MLPClassifier(hidden_layer_sizes = [node], alpha = .1,
                   random_state = 0,activation='relu',solver='adam').fit(X_train, y_train)
    print("Nodes: " + str(node))
    print('Accuracy of NN classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
    print('Accuracy of NN classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))
    print()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Nodes: 1
Accuracy of NN classifier on training set: 0.81
Accuracy of NN classifier on test set: 0.79



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Nodes: 5
Accuracy of NN classifier on training set: 0.90
Accuracy of NN classifier on test set: 0.93



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Nodes: 10
Accuracy of NN classifier on training set: 0.89
Accuracy of NN classifier on test set: 0.94

Nodes: 20
Accuracy of NN classifier on training set: 0.93
Accuracy of NN classifier on test set: 0.95

Nodes: 25
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.92



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [10]:
for layer1 in [1, 5, 10, 20, 25]:
    for layer2 in [1, 5, 10, 20, 25]:
        # training the data
        clf = MLPClassifier(hidden_layer_sizes = [layer1, layer2], alpha = .1,
                       random_state = 0, activation='relu',solver='adam').fit(X_train, y_train)
        print("Layer 1 Nodes: " + str(layer1))
        print("Layer 2 Nodes: " + str(layer2))
        print('Accuracy of NN classifier on training set: {:.2f}'
         .format(clf.score(X_train, y_train)))
        print('Accuracy of NN classifier on test set: {:.2f}'
         .format(clf.score(X_test, y_test)))
        print()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 1
Layer 2 Nodes: 1
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.79



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 1
Layer 2 Nodes: 5
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

Layer 1 Nodes: 10
Layer 2 Nodes: 5
Accuracy of NN classifier on training set: 0.91
Accuracy of NN classifier on test set: 0.92



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 10
Layer 2 Nodes: 10
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.92

Layer 1 Nodes: 10
Layer 2 Nodes: 20
Accuracy of NN classifier on training set: 0.90
Accuracy of NN classifier on test set: 0.90

Layer 1 Nodes: 10
Layer 2 Nodes: 25
Accuracy of NN classifier on training set: 0.87
Accuracy of NN classifier on test set: 0.86

Layer 1 Nodes: 20
Layer 2 Nodes: 1
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 20
Layer 2 Nodes: 5
Accuracy of NN classifier on training set: 0.81
Accuracy of NN classifier on test set: 0.80

Layer 1 Nodes: 20
Layer 2 Nodes: 10
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.91

Layer 1 Nodes: 20
Layer 2 Nodes: 20
Accuracy of NN classifier on training set: 0.94
Accuracy of NN classifier on test set: 0.92

Layer 1 Nodes: 20
Layer 2 Nodes: 25
Accuracy of NN classifier on training set: 0.89
Accuracy of NN classifier on test set: 0.89

Layer 1 Nodes: 25
Layer 2 Nodes: 1
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Layer 1 Nodes: 25
Layer 2 Nodes: 5
Accuracy of NN classifier on training set: 0.94
Accuracy of NN classifier on test set: 0.94

Layer 1 Nodes: 25
Layer 2 Nodes: 10
Accuracy of NN classifier on training set: 0.91
Accuracy of NN classifier on test set: 0.91

Layer 1 Nodes: 25
Layer 2 Nodes: 20
Accuracy of NN classifier on training set: 0.84
Accuracy of NN c

In [11]:
for layer1 in [1, 5, 10, 20, 25]:
    for layer2 in [1, 5, 10, 20, 25]:
        for layer3 in [1, 5, 10, 20, 25]:
            # training the data
            clf = MLPClassifier(hidden_layer_sizes = [layer1, layer2, layer3], alpha = .1,
                           random_state = 0, activation='relu',solver='adam').fit(X_train, y_train)
            print("Layer 1 Nodes: " + str(layer1))
            print("Layer 2 Nodes: " + str(layer2))
            print("Layer 3 Nodes: " + str(layer3))
            print('Accuracy of NN classifier on training set: {:.2f}'
             .format(clf.score(X_train, y_train)))
            print('Accuracy of NN classifier on test set: {:.2f}'
             .format(clf.score(X_test, y_test)))
            print()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 1
Layer 2 Nodes: 1
Layer 3 Nodes: 1
Accuracy of NN classifier on training set: 0.20
Accuracy of NN classifier on test set: 0.19



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 1
Layer 2 Nodes: 1
Layer 3 Nodes: 5
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

Layer 1 Nodes: 25
Layer 2 Nodes: 1
Layer 3 Nodes: 1
Accuracy of NN classifier on training set: 0.20
Accuracy of NN classifier on test set: 0.19



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 25
Layer 2 Nodes: 1
Layer 3 Nodes: 5
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Layer 1 Nodes: 25
Layer 2 Nodes: 1
Layer 3 Nodes: 10
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Layer 1 Nodes: 25
Layer 2 Nodes: 1
Layer 3 Nodes: 20
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Layer 1 Nodes: 25
Layer 2 Nodes: 1
Layer 3 Nodes: 25
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Layer 1 Nodes: 25
Layer 2 Nodes: 5
Layer 3 Nodes: 1
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Layer 1 Nodes: 25
Layer 2 Nodes: 5
Layer 3 Nodes: 5
Accuracy of NN classifier on training set: 0.85
Accuracy of NN classifier on test set: 0.86

Layer 1 Nodes: 25
Layer 2 Nodes: 5
Layer 3 Nodes: 10
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.94

Layer 1 Nodes: 25
Layer 2 Nodes: 5
Layer 3 Nodes: 20
Accuracy of NN classifier on training set: 0.81
Accuracy of NN classifier on test set: 0.84

Layer 1 Nodes: 25
Layer 2 Nodes: 5
Layer 3 Nodes: 25
Accuracy of NN classifier on training set: 0.90
Accuracy of NN classifier on test set: 0.91

Layer 1 Nodes: 25
Layer 2 Nodes: 10
Layer 3 Nodes: 1
Accuracy of NN classifier on training set: 0.35
Accuracy of NN classifier on test set: 0.34

Layer 1 Nodes: 25
Layer 2 Nodes: 10
Layer 3 Nodes: 5
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Layer 1 Nodes: 25
Layer 2 Nodes: 10
Layer 3 Nodes: 10
Accuracy of NN classifier on training set: 0.91
Accuracy of NN classifi

In [12]:
for this_activation in ['identity','logistic','tanh','relu']:

    clf = MLPClassifier(hidden_layer_sizes = [10,10],solver='adam',
                          alpha=0.1,activation =this_activation,
                         random_state = 0).fit(X_train,y_train)
    
    
    print("Activation: " + str(this_activation))
    print('Accuracy of NN classifier on training set: {:.2f}'
             .format(clf.score(X_train, y_train)))
    print('Accuracy of NN classifier on test set: {:.2f}'
             .format(clf.score(X_test, y_test)))
    print()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Activation: identity
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.92



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Activation: logistic
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Activation: tanh
Accuracy of NN classifier on training set: 0.81
Accuracy of NN classifier on test set: 0.84

Activation: relu
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.92



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
for this_alpha in [.1, .5, 1,5,10,20,50,100]:
    # training the data
    clf = MLPClassifier(hidden_layer_sizes = [10, 10], alpha = this_alpha,
                   random_state = 0, activation='relu',solver='adam').fit(X_train_scaled, y_train)
    print('Alpha: ' + str(this_alpha))
    print('Accuracy of NN classifier on training set: {:.2f}'
         .format(clf.score(X_train_scaled, y_train)))
    print('Accuracy of NN classifier on test set: {:.2f}'
         .format(clf.score(X_test_scaled, y_test)))
    print()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Alpha: 0.1
Accuracy of NN classifier on training set: 0.97
Accuracy of NN classifier on test set: 0.95



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Alpha: 0.5
Accuracy of NN classifier on training set: 0.97
Accuracy of NN classifier on test set: 0.95



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Alpha: 20
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Alpha: 50
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Alpha: 100
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [14]:
for this_solver in ['lbfgs', 'sgd', 'adam']:

    clf = MLPClassifier(hidden_layer_sizes = [10,10],solver=this_solver,
                          alpha=0.1, random_state = 0, activation ='relu').fit(X_train,y_train)
    
    
    print("Solver " + str(this_solver))
    print('Accuracy of NN classifier on training set: {:.2f}'
             .format(clf.score(X_train, y_train)))
    print('Accuracy of NN classifier on test set: {:.2f}'
             .format(clf.score(X_test, y_test)))
    print()

Solver lbfgs
Accuracy of NN classifier on training set: 0.79
Accuracy of NN classifier on test set: 0.79

Solver sgd
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.80

Solver adam
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.92



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [15]:
for initial_value in [.01,.1,1,10]:    
    for rate in ['constant', 'invscaling', 'adaptive']:
        clf = MLPClassifier(hidden_layer_sizes = [10,10],solver='sgd',
                              alpha=0.1, random_state = 0, activation ='relu', learning_rate=rate, learning_rate_init=initial_value).fit(X_train,y_train)


        print("Learning Rate inital: " + str(initial_value))
        print("Learning Rate: " + str(rate))
        print('Accuracy of NN classifier on training set: {:.2f}'
                 .format(clf.score(X_train, y_train)))
        print('Accuracy of NN classifier on test set: {:.2f}'
                 .format(clf.score(X_test, y_test)))
        print()

Learning Rate inital: 0.01
Learning Rate: constant
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Learning Rate inital: 0.01
Learning Rate: invscaling
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Learning Rate inital: 0.01
Learning Rate: adaptive
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Learning Rate inital: 0.1
Learning Rate: constant
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Learning Rate inital: 0.1
Learning Rate: invscaling
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Learning Rate inital: 0.1
Learning Rate: adaptive
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 0.81

Learning Rate inital: 1
Learning Rate: constant
Accuracy of NN classifier on training set: 0.80
Accuracy of NN classifier on test set: 

In [16]:
for this_iter in [10,100,200,500,1000]:

    clf = MLPClassifier(hidden_layer_sizes = [10,10],solver=this_solver,
                          alpha=0.1, random_state = 0, activation ='relu', max_iter=this_iter).fit(X_train,y_train)
    
    
    print("Iterations: " + str(this_iter))
    print('Accuracy of NN classifier on training set: {:.2f}'
             .format(clf.score(X_train, y_train)))
    print('Accuracy of NN classifier on test set: {:.2f}'
             .format(clf.score(X_test, y_test)))
    print()

Iterations: 10
Accuracy of NN classifier on training set: 0.65
Accuracy of NN classifier on test set: 0.70



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iterations: 100
Accuracy of NN classifier on training set: 0.88
Accuracy of NN classifier on test set: 0.90



/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iterations: 200
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.92

Iterations: 500
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.90

Iterations: 1000
Accuracy of NN classifier on training set: 0.92
Accuracy of NN classifier on test set: 0.90

